In [ ]:
# Benchmarks
# Testing core functionality, adhere to paper as closely as possible:
# no translation of hashable node labels to integers, just use graph 
# labels DIRECTLY 

# Generate erdos_renyi_graphs:
# Can have a specified number of nodes and probability of edges being made

In [ ]:
import networkx as nx
import numpy as np
import gsgmorph.matrix_form
import timeit
import matplotlib.pyplot as plt
import csv

In [ ]:
# increase number of nodes over time
#main_hm_times = []
#main_gsgp_times = []
rng = np.random.default_rng()

with open('node_benchmark.csv', 'w', newline='') as csvfile:
    fieldnames = ['num_nodes', 'gsgmorph', 'hua', 'qubo_equal']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # go from 1 to 100 nodes, with 0.5 probability for edge formation
    for n in range(1, 100):
        g1 = nx.erdos_renyi_graph(n, 0.5, seed=0)

        
        #gsgmorph_times = []
        #hua_times = []

        print("\rNodes: {0}\r".format(n), end='')

        # Keep the edges the same but shuffle the nodes around
        perm = rng.permutation(g1.nodes())
        mapping = {i: v for i, v in zip(g1.nodes(), perm)}
        g2 = nx.relabel_nodes(g1, mapping, copy=True)
        
        def gsgm_graph_isomorphism():
            gsgmorph.matrix_form.integer_graph_isomorphism(g1,g2)
        def hua_graph_isomorphism():
            gsgmorph.matrix_form.hua_graph_isomorphism(g1,g2)
        # `timeit` arguments: https://stackoverflow.com/questions/56763416/what-is-diffrence-between-number-and-repeat-in-python-timeit
        # number: run the program back-to-back 
        # repeat: repeat * number (see line above)
        gsgmorph_time = min(timeit.repeat(gsgm_graph_isomorphism, repeat=2, number=4))
        hua_time = min(timeit.repeat(hua_graph_isomorphism, repeat=2, number=4))

        gsgmorph_matrix = gsgmorph.matrix_form.integer_graph_isomorphism(g1, g2) 
        hua_matrix = gsgmorph.matrix_form.hua_graph_isomorphism(g1, g2)
        matrix_equal = ""
        
        if(gsgmorph_matrix == hua_matrix):
            matrix_equal = "TRUE"
        else:
            matrix_equal = "FALSE"
        writer.writerow({'num_nodes':n, 
                         'gsgmorph': gsgmorph_time,
                         'hua': hua_time,
                         'qubo_equal':matrix_equal})